In [2]:
#Import Useful Modules
from pyatmos import download_sw_nrlmsise00, read_sw_nrlmsise00 #atmosphere data
from pyatmos import nrlmsise00
import sympy as sym
import matplotlib.pyplot as plt
import numpy as np
from scipy import integrate
import time
import pandas as pd

#Requires ipympl
#%matplotlib widget

import warnings
warnings.filterwarnings("ignore")

The EOP file 'finals2000A.all' in C:\Users\Brody/src/iers/ is already the latest.
The Leap Second file 'Leap_Second.dat' in C:\Users\Brody/src/iers/ is already the latest.


In [3]:
# Update space weather data
swfile = download_sw_nrlmsise00() 
swdata = read_sw_nrlmsise00(swfile) 

The Space Weather file 'SW-All.csv' in C:\Users\Brody/src/sw-data/ is already the latest.


In [4]:
M_dict = {
    'He': 4.002602,
    'O': 15.999,
    'N2': 28.014,
    'O2': 31.998,
    'Ar': 39.948,
    'H': 1.00784,
    'N': 14.007,
    'ANM O': 15.999,
}
g_dict = {
    'He': 1.66,
    'O': 1.66,
    'N2': 1.40,
    'O2': 1.40,
    'Ar': 1.66,
    'H': 1.66,
    'N': 1.66,
    'ANM O': 1.66,
}
Av_kg = 6.023e23*1e3 # 1/k mol
R_ideal = 8.314*1e3 #mJ / mol K

In [5]:
#Earth Parameters
r_e = 6.371*1e6 #m
G_e = 3.986*1e14 #m3⋅s−2

In [6]:
t = '2023-11-20 12:00:00' # time(UTC) 
#lat,lon,alt = 25,102,86 # latitude, longitude in [degree], and altitude in [km]

def speed_of_sound(lat, lon, alt, t):
    nrl00 = nrlmsise00(t, (lat, lon, alt), swdata)
    
    t_d = 0
    n_c = 0
    d_c = 0
    for spec in nrl00.nd:
        t_d += nrl00.nd[spec]
        n_c += nrl00.nd[spec]*g_dict[spec]/(g_dict[spec]-1)
        d_c += nrl00.nd[spec]/(g_dict[spec]-1)

    M_net = (nrl00.rho*Av_kg/t_d)
    R_net = R_ideal/M_net
    gamma_net = n_c/d_c
    cs_calc = np.sqrt(gamma_net*R_net*nrl00.T)
    
    return cs_calc, gamma_net, nrl00.rho

In [7]:
#Create symbols to use in equations
#Note: +x is defined as the windward direction
x_s, y_s, z_s = sym.symbols('x y z')
vs = sym.symbols('y z')
R_s, L_s, a_s, M_s = sym.symbols('R L a M')

In [29]:
#----------- SET VEHICLE BODY PARAMETERS ----------
#Radius
R = 2.56 #m
#Length factor
L = 0.3*R
#Mass
m = 9300 #kg
#Body shape
body_shape = 'ellipsoid'
    
#----------- ADDITIONAL BODY PARAMETERS -------------
#Exclude edges of the surface from the integration domain to prevent errors
bound_sf = (1-1e-15)

In [22]:
#----------- SET VEHICLE BODY PARAMETERS ----------
#Radius
R = 0.27 #m
#Length factor
L = 1.8
#Mass
m = 350 #kg
#Body shape
body_shape = 'cone'
    
#----------- ADDITIONAL BODY PARAMETERS -------------
#Exclude edges of the surface from the integration domain to prevent errors
bound_sf = (1-1e-15)

In [30]:
##----------- SET FLIGHT VEHICLE SHAPE EQUATIONS #-----------

if body_shape == 'ellipsoid':
    #----------- SET ELLIPSOID EQUATIONS ----------------
    #Windward surface of the ellipsoid
    s = (-(2*z_s*(L_s**2-R_s**2)*sym.cos(a_s)*sym.sin(a_s))-sym.sqrt((2*(L_s**2-R_s**2)*sym.cos(a_s)*sym.sin(a_s)*z_s)**2-4*(R_s**2*sym.cos(a_s)**2+L_s**2*sym.sin(a_s)**2)*(z_s**2*(R_s**2*sym.sin(a_s)**2+L_s**2*sym.cos(a_s)**2)+L_s**2*(y_s**2-R_s**2))))/(2*(R_s**2*sym.cos(a_s)**2+L_s**2*sym.sin(a_s)**2))
    #Upper surface curve when set equal to zero, symmetric across horizontal axis
    bu_f = (2*(L_s**2-R_s**2)*sym.cos(a_s)*sym.sin(a_s)*z_s)**2-4*(R_s**2*sym.cos(a_s)**2+L_s**2*sym.sin(a_s)**2)*(z_s**2*(R_s**2*sym.sin(a_s)**2+L_s**2*sym.cos(a_s)**2)+L_s**2*(y_s**2-R_s**2))

elif body_shape == 'cone':
    #----------- SET CONE EQUATIONS ----------------
    #Windward surface of the ellipsoid
    s = ((2*R_s**2*(z_s-L_s*sym.sin(a_s))*sym.cos(a_s)*sym.sin(a_s)+2*(z_s-L_s*sym.sin(a_s))*sym.cos(a_s)*sym.sin(a_s))+sym.sqrt((-2*R_s**2*(z_s-L_s*sym.sin(a_s))*sym.cos(a_s)*sym.sin(a_s)-2*(z_s-L_s*sym.sin(a_s))*sym.cos(a_s)*sym.sin(a_s))**2-4*(R_s**2*sym.cos(a_s)**2-sym.sin(a_s)**2)*(R_s**2*(z_s-L_s*sym.sin(a_s))**2*sym.sin(a_s)**2-(z_s-L_s*sym.sin(a_s))**2*sym.cos(a_s)**2-y_s**2)))/(2*(R_s**2*sym.cos(a_s)**2-sym.sin(a_s)**2))
    #Upper surface curve when set equal to zero, symmetric across horizontal axis
    bu_f = sym.cos(a_s)*sym.sqrt(L_s**2*R_s**2-y_s**2)-z_s

In [31]:
#--------- USEFUL CALCULATION QUANTITIES -----------
#Partial derivatives of the surface function
py = sym.diff(s, y_s)
pz = sym.diff(s, z_s)

#Solution for the upper boundary of integration
bu = sym.solve(bu_f, z_s)
bu = bu[len(bu)-1]

#Square of the normal component of the surface, n_x^2
nx2 = 1/(1+py**2+pz**2)

#Integrand for calculating coeficient of lift
nx2pz = nx2*pz

In [32]:
#----- FUNCTION FOR GETTING C_d and C_l FROM AOA -------
def coeff_aoa(aoa, area_scaled = True):
    #Convert from degrees to radians
    aoa = aoa*np.pi/180
    
    #------- CREATE FAST FUNCTIONS FOR INTEGRATION ---------
    #Upper edge height from  pitching axis as function of dist along that axis
    #For a given body geometry and angle of attack
    fvbu = bound_sf*bu.subs(L_s,L).subs(R_s,R).subs(a_s,aoa).evalf()
    syms = [sym.parse_expr(item) for item in [str(fvbu)]]
    f_bu = [sym.lambdify(y_s, f) for f in syms]

    #C_d integrand as a function of position
    #For a given body geometry and angle of attack
    fvnx2 = nx2.subs(L_s,L).subs(R_s,R).subs(a_s,aoa).evalf()
    syms = [sym.parse_expr(item) for item in [str(fvnx2)]]
    f_nx2 = [sym.lambdify(vs, f) for f in syms]

    #C_l integrand as a function of position
    #For a given body geometry and angle of attack
    fnx2pz = nx2pz.subs(L_s,L).subs(R_s,R).subs(a_s,aoa).evalf()
    syms = [sym.parse_expr(item) for item in [str(fnx2pz)]]
    f_nx2pz = [sym.lambdify(vs, f) for f in syms]
    
    #------- INTEGRATIONS ---------
    #Projected cross sectional area (half)
    A = integrate.dblquad(lambda z, y: 1, 0, R, lambda y: -[f(y) for f in f_bu][0], lambda y: [f(y) for f in f_bu][0])[0]
    
    #C_d (divided by maximum C_p)
    C_d_Cpm_i = integrate.dblquad(lambda z, y: [f(y, z) for f in f_nx2][0] , 0, R, lambda y: -[f(y) for f in f_bu][0], lambda y: [f(y) for f in f_bu][0])[0]

    #C_l (divided by maximum C_p)
    C_l_Cpm_i = integrate.dblquad(lambda z, y: [f(y, z) for f in f_nx2pz][0] , 0, R, lambda y: -[f(y) for f in f_bu][0], lambda y: [f(y) for f in f_bu][0])[0]
    
    if not area_scaled:
        #Return purely the aero coefficients
        C_d_Cpm_i = C_d_Cpm_i/A
        C_l_Cpm_i = C_l_Cpm_i/A

        #Otherwise return aero coefficients * A/2 for easier computation
        
    return C_d_Cpm_i, C_l_Cpm_i

In [33]:
#----- FUNCTION FOR GETTING INSTANTANEOUS AERO PARAMETERS -------
def instant_coeff(C_d_Cpm, C_l_Cpm, M, gamma):
    #Instantaneous coeffcient of max pressure
    C_p_max = (2/(gamma*M**2))*((((gamma+1)**2*M**2)/(4*gamma*M**2-2*gamma+2))**(gamma/(gamma-1))*((1-gamma+2*gamma*M**2)/(gamma+1))-1)

    #Instantaneous coeffcient of drag
    C_d = C_d_Cpm*C_p_max

    #Instantaneous coeffcient of lift
    C_l = C_l_Cpm*C_p_max
    
    return C_p_max, C_d, C_l

In [34]:
def basic_step(r_vec, v_vec, dt=1.0, bank = 0.0):
    #------- FLIGHT INSTANTANEOUS CONDITION --------
    v_mag = np.linalg.norm(v_vec)
    z_alt = np.linalg.norm(r_vec)-r_e
    d_vec = r_vec/np.linalg.norm(r_vec)
    d_vel = (v_vec/v_mag)
    bank = bank*(np.pi/180)

    #Global Position Coordinates
    lat_int = 90-np.arccos(d_vec[2])*180/np.pi
    long_int = np.arccos(d_vec[0]/np.sin((90-lat_int)*np.pi/180))*180/np.pi

    cs_calc, gamma_calc, rho_calc = speed_of_sound(lat_int, long_int, z_alt/1e3, t)

    M_v = v_mag/cs_calc
    gamma_v = gamma_calc
    
    if M_v < 5 or z_alt<0:
        return r_vec, v_vec, False

    #Instantaneous aerodynamic coefficients
    C_p_max, C_d, C_l = instant_coeff(C_d_Cpm, C_l_Cpm , M=M_v, gamma=gamma_v)

    D_f = C_d*rho_calc*v_mag*v_mag
    D_acc = D_f/m

    L_f = C_l*rho_calc*v_mag*v_mag
    L_acc = L_f/m

    W_acc = G_e/(z_alt+r_e)**2

    r_vec += dt * v_vec
    v_vec += -dt * W_acc * d_vec
    v_vec += -dt * D_acc * d_vel
    
    #Lift Caclulations
        #Zero bank lift direction
    d_l0b = np.cross(d_vel, np.cross(d_vel, d_vec))
    d_lift = np.cos(bank)*d_l0b + np.sin(bank) * np.cross(d_vel, d_l0b)
    v_vec += dt * L_acc * d_lift
    
    acc_g = np.sqrt(D_acc**2 + (L_acc)**2)
        
    df.loc[len(df.index)] = [df['time'].iloc[-1]+dt, r_vec[0], r_vec[1], r_vec[2], v_vec[0], v_vec[1], v_vec[2], acc_g, M_v] 
    
    return r_vec, v_vec, True

In [35]:
#------- INITIALIZE FLIGHT CONDITIONS --------
lat_i = 0
long_i = 0
z_i = 400000*0.3048
entry_rot = -2*np.pi/180 #5.65
v_entry_mag = 8000 #10987

dt_mult = 1.

In [28]:
for k in np.linspace(0,8,5):
    if k == 0:
        k=0.01
    C_d_Cpm, C_l_Cpm = coeff_aoa(k, area_scaled = True)
    for j in np.linspace(0, 180, 37):

        z_vec = np.array([z_i*np.sin((90-lat_i)*np.pi/180)*np.cos(long_i*np.pi/180), z_i*np.sin((90-lat_i)*np.pi/180)*np.sin(long_i*np.pi/180), z_i*np.cos((90-lat_i)*np.pi/180)])
        r_vec = (np.linalg.norm(z_vec)+r_e)*z_vec/np.linalg.norm(z_vec)
        v_vec = np.array([v_entry_mag*np.sin(entry_rot), v_entry_mag*np.cos(entry_rot), 0.0])

        df = pd.DataFrame({'time':[0], 'r_x':[r_vec[0]], 'r_y':[r_vec[1]], 'r_z':[r_vec[1]], 'v_x':[v_vec[0]], 'v_y':[v_vec[1]], 'v_z':[v_vec[1]], 'a':[0], 'M':[0]})

        for i in range(int(50000*dt_mult)):
            r_vec, v_vec, hyper = basic_step(r_vec, v_vec, dt=1./dt_mult, bank = j)
            if not hyper:
                break
        file_name = 'data/cones_leo_a'+str(k)+'_b'+str(j)
        file_name = file_name.replace(".", "-")+'.csv'
        df.to_csv(file_name)
        print('aoa: '+str(k)+'   ,   bank:'+str(j))

aoa: 0.01   ,   bank:0.0
aoa: 0.01   ,   bank:5.0
aoa: 0.01   ,   bank:10.0
aoa: 0.01   ,   bank:15.0
aoa: 0.01   ,   bank:20.0
aoa: 0.01   ,   bank:25.0
aoa: 0.01   ,   bank:30.0
aoa: 0.01   ,   bank:35.0
aoa: 0.01   ,   bank:40.0
aoa: 0.01   ,   bank:45.0
aoa: 0.01   ,   bank:50.0
aoa: 0.01   ,   bank:55.0
aoa: 0.01   ,   bank:60.0
aoa: 0.01   ,   bank:65.0
aoa: 0.01   ,   bank:70.0
aoa: 0.01   ,   bank:75.0
aoa: 0.01   ,   bank:80.0
aoa: 0.01   ,   bank:85.0
aoa: 0.01   ,   bank:90.0
aoa: 0.01   ,   bank:95.0
aoa: 0.01   ,   bank:100.0
aoa: 0.01   ,   bank:105.0
aoa: 0.01   ,   bank:110.0
aoa: 0.01   ,   bank:115.0
aoa: 0.01   ,   bank:120.0
aoa: 0.01   ,   bank:125.0
aoa: 0.01   ,   bank:130.0
aoa: 0.01   ,   bank:135.0
aoa: 0.01   ,   bank:140.0
aoa: 0.01   ,   bank:145.0
aoa: 0.01   ,   bank:150.0
aoa: 0.01   ,   bank:155.0
aoa: 0.01   ,   bank:160.0
aoa: 0.01   ,   bank:165.0
aoa: 0.01   ,   bank:170.0
aoa: 0.01   ,   bank:175.0
aoa: 0.01   ,   bank:180.0
aoa: 2.0   ,   bank:0.0